# Andmete puhastamine

Siin töötleme varasemas etapis kogutud toorandmeid (`toorandmed_aasta.csv`).

**Sisend:** `toorandmed_aasta.csv`

**Väljund:** `andmed_aasta.csv`

### Samm 0: Teekide laadimine ja failiteede seadistamine
Laeme vajalikud Pythoni teegid ja fikseerime sisend- ja väljundfailide asukohad. Loeme sisse puhastamata andmed.

In [ ]:
import pandas as pd
import json
import numpy as np
import os

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)

INPUT_FILE = 'toorandmed_aasta.csv'
OUTPUT_FILE = 'andmed_aasta.csv'

if os.path.exists(INPUT_FILE):
    df = pd.read_csv(INPUT_FILE, low_memory=False)
    print(f"Toorandmed loetud. Andmestiku suurus: {df.shape[0]} rida, {df.shape[1]} veergu.")
else:
    raise FileNotFoundError(f"Viga: Sisendfaili '{INPUT_FILE}' ei leitud! Kontrolli failiteed.")

df.head(2)

### Samm 1: Eemaldame andmed, mida me ei taha

Filtreerime välja read, mis ei sobi meie rakenduse jaoks:
- **`additional_info__duration_in_semesters`** — viskame ära read, kus see on suurem kui 1
- **`version__target__study_type__et`** — jätame ainult päevaõppe ained
- **`general__type__et`** — jätame ainult tavalised ained (välistame praktikad, kaitsmised, kompleksained)

In [ ]:
print(f"Ridu enne filtreerimist: {len(df)}")

if 'additional_info__duration_in_semesters' in df.columns:
    mask_duration = df['additional_info__duration_in_semesters'] <= 1
    df = df[mask_duration]
    print(f"Pärast kestuse filtrit (<=1 semester): {len(df)} rida")

if 'version__target__study_type__et' in df.columns:
    mask_study_type = df['version__target__study_type__et'].isna() | df['version__target__study_type__et'].str.contains('päevaõ', case=False, na=False)
    df = df[mask_study_type]
    print(f"Pärast õppetüübi filtrit (päevaõpe): {len(df)} rida")

if 'general__type__et' in df.columns:
    print(f"general__type__et väärtused enne filtrit:\n{df['general__type__et'].value_counts()}")
    mask_type = df['general__type__et'].str.contains('Tavaline', case=False, na=False)
    df = df[mask_type]
    print(f"\nPärast aine tüübi filtrit (Tavaline aine): {len(df)} rida")

print(f"\nLõplik ridade arv pärast filtreerimist: {len(df)}")

### Samm 2: Puhastatud veergude lisamine

Valime välja huvipakkuvad veerud ning puhastame need. Eelistame alati `version__` prefiksiga veerge, sest need kajastavad tegelikku aineprogrammi. Kui see puudub, võtame üldise info.

In [ ]:
def resolve_fields(df):
    """
    Loob uued puhtad veerud, eelistades versiooni-põhist infot.
    Tagastab täiendatud DataFrame'i.
    """
    merge_mapping = [
        ('nimi_et', 'title__et', 'version__title__et'),
        ('nimi_en', 'title__en', 'version__title__en'),
        ('eap', 'credits', 'version__credits'),
        ('kirjeldus_et', 'overview__description__et', 'version__overview__description__et'),
        ('kirjeldus_en', 'overview__description__en', 'version__overview__description__en'),
        ('oppivaaljundid_en', 'overview__learning_outcomes_text_en', 'version__overview__learning_outcomes_text_en'),
        ('oppivaaljundid_et', 'overview__learning_outcomes_text_et', 'version__overview__learning_outcomes_text_et'),
        ('keel_et', 'version__target__language__et', None),
        ('keel_en', 'version__target__language__en', None),
    ]

    rename_mapping = [
        ('aine_kood', 'code'),
        ('semester', 'version__target__semester__et'),
        ('hindamisskaala', 'additional_info__assessment_scale__et'),
        ('linn', 'version__target__course_main_structural_unit__city'),
        ('hindamine', 'version__grading__grade_evaluation__et'),
        ('taiskoormus', 'additional_info__is_continuous_learning_course'),
        ('kestus_semestrites', 'additional_info__duration_in_semesters'),
        ('oppe_tuup', 'version__target__study_type__et'),
        ('aine_tuup', 'general__type__et'),
        ('eeldusained_json', 'additional_info__prerequisites'),
        ('oppeaste_json', 'version__additional_info__study_levels'),
    ]

    print("Alustan veergude filtreerimist ja ühendamist...")

    for new_col, base, version in merge_mapping:
        base_exists = base in df.columns if base else False
        ver_exists = version in df.columns if version else False
        if base_exists and ver_exists:
            df[new_col] = df[version].fillna(df[base])
        elif ver_exists:
            df[new_col] = df[version]
        elif base_exists:
            df[new_col] = df[base]
        else:
            df[new_col] = np.nan

    for new_col, source in rename_mapping:
        if source in df.columns:
            df[new_col] = df[source]
        else:
            df[new_col] = np.nan

    return df

df_resolved = resolve_fields(df.copy())

print(f"Samm 2: Puhastatud veergude lisamine tehtud.")
print(f"\nPuuduvate väärtuste arv uutes veergudes:")
new_cols = ['aine_kood', 'nimi_et', 'nimi_en', 'eap', 'semester', 'kirjeldus_et', 'kirjeldus_en',
            'oppivaaljundid_en', 'oppivaaljundid_et', 'keel_et', 'keel_en',
            'hindamisskaala', 'linn', 'hindamine', 'taiskoormus',
            'kestus_semestrites', 'oppe_tuup', 'aine_tuup',
            'eeldusained_json', 'oppeaste_json']
print(df_resolved[new_cols].isnull().sum())

### Samm 3: JSON väljade parsimine

Parsime 2 JSON veergu:
- **Eeldusained** — eraldame ainete koodid ja nimetused
- **Õppeaste** — eraldame taseme (bakalaureus, magister, doktor)

In [ ]:
def parse_json_safe(json_str):
    """Teisendab JSON-stringi turvaliselt Pythoni objektiks."""
    if pd.isna(json_str) or json_str == '':
        return None
    try:
        return json.loads(json_str)
    except (json.JSONDecodeError, TypeError):
        return None

def extract_prerequisites(json_str):
    """Eraldab eeldusainete koodid ja nimed."""
    data = parse_json_safe(json_str)
    if not data: return None
    courses = []
    for item in data:
        if not isinstance(item, dict): continue
        code = item.get('code', '')
        title_obj = item.get('title', {})
        title = title_obj.get('et', title_obj.get('en', '')) if isinstance(title_obj, dict) else ''
        if code and title:
            courses.append(f"{code} - {title}")
        for alt in item.get('alternatives', []):
            if not isinstance(alt, dict): continue
            alt_code = alt.get('code', '')
            alt_title_obj = alt.get('title', {})
            alt_title = alt_title_obj.get('et', alt_title_obj.get('en', '')) if isinstance(alt_title_obj, dict) else ''
            if alt_code and alt_title:
                courses.append(f"{alt_code} - {alt_title}")
    return "; ".join(courses) if courses else None

def extract_study_levels(json_str):
    """Eraldab õppeastme nimetused eesti keeles."""
    data = parse_json_safe(json_str)
    if not data: return None
    levels = []
    for item in data:
        if not isinstance(item, dict): continue
        level_et = item.get('et', item.get('en', ''))
        if level_et:
            levels.append(level_et)
    return ", ".join(levels) if levels else None

print("Ekstraheerime JSON väljadest infot...")
df_resolved['eeldusained'] = df_resolved['eeldusained_json'].apply(extract_prerequisites)
df_resolved['oppeaste'] = df_resolved['oppeaste_json'].apply(extract_study_levels)

print("Samm 3: JSON töötlemine valmis.\n")
print(df_resolved[['aine_kood', 'eeldusained', 'oppeaste']].dropna(subset=['eeldusained']).head(3))

### Samm 4: Info koondamine mitmest allikast

Koondame hindamisinfo mitmest veerust üheks tekstiks.

In [ ]:
grading_source_cols = [
    ('Miinimumnõuded', 'version__grading__grade_preconditions__et'),
    ('Hindamismeetod', 'version__grading__grade_evaluation__et')
]

def combine_grading_info(row):
    """Koondab hindamisinfo mitmest veerust üheks tekstiks."""
    parts = []
    for label, col in grading_source_cols:
        val = row.get(col)
        if pd.notna(val) and str(val).strip():
            parts.append(f"{label}: {str(val).strip()}")
    return "\n".join(parts) if parts else None

df_resolved['hindamine_info'] = df_resolved.apply(combine_grading_info, axis=1)
print("Samm 4: Hindamisinfo koondatud.")
print(f"Puuduvaid väärtusi: {df_resolved['hindamine_info'].isna().sum()}")
print(f"\nNäide:")
sample = df_resolved['hindamine_info'].dropna().iloc[0]
print(sample[:300])

### Samm 5: Lõplik viimistlus

Valime lõplikud veerud.

In [ ]:
final_cols = [
    'aine_kood', 'nimi_et', 'nimi_en', 'eap', 'semester',
    'keel_et', 'keel_en', 'linn', 'oppeaste', 'hindamisskaala',
    'kirjeldus_et', 'kirjeldus_en',
    'oppivaaljundid_et', 'oppivaaljundid_en',
    'hindamine_info', 'eeldusained',
    'taiskoormus', 'kestus_semestrites', 'oppe_tuup', 'aine_tuup'
]

existing_cols = [c for c in final_cols if c in df_resolved.columns]
df_final = df_resolved[existing_cols].copy()

print(f"Lõplik ridade arv: {len(df_final)}, lõplik veergude arv: {len(df_final.columns)}")
print(f"Veerud: {list(df_final.columns)}")
print("Samm 5: Lõplik andmestik koostatud.")

### Samm 6: Salvestamine ja kontroll

In [ ]:
df_final.to_csv(OUTPUT_FILE, index=False)
print(f"Andmestik salvestatud faili '{OUTPUT_FILE}'.\n")

print("=" * 60)
print("PUUDUVATE VÄÄRTUSTE HULK IGAS VEERUS")
print("=" * 60)
missing = df_final.isnull().sum()
total = len(df_final)
missing_pct = (missing / total * 100).round(1)
missing_df = pd.DataFrame({'Puuduvaid': missing, 'Protsent (%)': missing_pct})
print(missing_df)
print()

In [ ]:
print("=" * 60)
print("KATEGOORILISTE TUNNUSTE ENIM LEVINUD VÄÄRTUSED")
print("=" * 60)

categorical_cols = ['keel_et', 'semester', 'oppeaste', 'hindamisskaala', 'linn',
                    'taiskoormus', 'kestus_semestrites', 'oppe_tuup', 'aine_tuup']

for col in categorical_cols:
    if col in df_final.columns:
        print(f"\n--- {col} ---")
        print(df_final[col].value_counts().head(5))
        print()

In [ ]:
print("=" * 60)
print("AINE: SISSEJUHATUS ANDMETEADUSESSE")
print("=" * 60)

mask = df_final['nimi_et'].str.contains('Sissejuhatus andmeteadusesse', case=False, na=False)
sissejuhatus = df_final[mask]

if len(sissejuhatus) == 0:
    print("Ainet 'Sissejuhatus andmeteadusesse' ei leitud! Proovime laiema otsinguga...")
    mask = df_final['nimi_et'].str.contains('andmeteadus', case=False, na=False)
    sissejuhatus = df_final[mask]

if len(sissejuhatus) > 0:
    row = sissejuhatus.iloc[0]
    line = chr(9472) * 50
    print(f"\n{line}")
    print(f"  Aine kood:        {row.get('aine_kood', 'N/A')}")
    print(f"  Nimi (ET):        {row.get('nimi_et', 'N/A')}")
    print(f"  Nimi (EN):        {row.get('nimi_en', 'N/A')}")
    print(f"  EAP:              {row.get('eap', 'N/A')}")
    print(f"  Semester:         {row.get('semester', 'N/A')}")
    print(f"  Keel:             {row.get('keel_et', 'N/A')}")
    print(f"  Linn:             {row.get('linn', 'N/A')}")
    print(f"  Õppeaste:         {row.get('oppeaste', 'N/A')}")
    print(f"  Hindamisskaala:   {row.get('hindamisskaala', 'N/A')}")
    print(f"  Täiskoormus:      {row.get('taiskoormus', 'N/A')}")
    print(f"  Kestus (sem.):    {row.get('kestus_semestrites', 'N/A')}")
    print(f"  Õppe tüüp:        {row.get('oppe_tuup', 'N/A')}")
    print(f"  Aine tüüp:        {row.get('aine_tuup', 'N/A')}")
    print(f"  Eeldusained:      {row.get('eeldusained', 'N/A')}")
    print(f"{line}")
    print(f"\n  Kirjeldus (ET):")
    print(f"  {row.get('kirjeldus_et', 'N/A')}")
    print(f"\n  Kirjeldus (EN):")
    print(f"  {row.get('kirjeldus_en', 'N/A')}")
    print(f"\n  Õpiväljundid (ET):")
    print(f"  {row.get('oppivaaljundid_et', 'N/A')}")
    print(f"\n  Õpiväljundid (EN):")
    print(f"  {row.get('oppivaaljundid_en', 'N/A')}")
    print(f"\n  Hindamise info:")
    print(f"  {row.get('hindamine_info', 'N/A')}")
    print(f"{line}")
else:
    print("Ainet ei leitud andmestikust.")

In [ ]:
print("=" * 60)
print("TUNNUSE 'KIRJELDUS' LOOMINE JA STATISTIKA")
print("=" * 60)

def build_kirjeldus(row):
    """Ühendab kõik aine veerud üheks suureks tekstiks RAGi jaoks."""
    parts = []
    if pd.notna(row.get('nimi_et')): parts.append(f"Aine: {row['nimi_et']}")
    if pd.notna(row.get('nimi_en')): parts.append(f"Course: {row['nimi_en']}")
    if pd.notna(row.get('aine_kood')): parts.append(f"Kood: {row['aine_kood']}")
    if pd.notna(row.get('eap')): parts.append(f"EAP: {row['eap']}")
    if pd.notna(row.get('semester')): parts.append(f"Semester: {row['semester']}")
    if pd.notna(row.get('keel_et')): parts.append(f"Keel: {row['keel_et']}")
    if pd.notna(row.get('linn')): parts.append(f"Linn: {row['linn']}")
    if pd.notna(row.get('oppeaste')): parts.append(f"Õppeaste: {row['oppeaste']}")
    if pd.notna(row.get('hindamisskaala')): parts.append(f"Hindamisskaala: {row['hindamisskaala']}")
    if pd.notna(row.get('kestus_semestrites')): parts.append(f"Kestus semestrites: {row['kestus_semestrites']}")
    if pd.notna(row.get('oppe_tuup')): parts.append(f"Õppe tüüp: {row['oppe_tuup']}")
    if pd.notna(row.get('aine_tuup')): parts.append(f"Aine tüüp: {row['aine_tuup']}")
    if pd.notna(row.get('kirjeldus_et')): parts.append(f"Kirjeldus: {row['kirjeldus_et']}")
    if pd.notna(row.get('kirjeldus_en')): parts.append(f"Description: {row['kirjeldus_en']}")
    if pd.notna(row.get('oppivaaljundid_et')): parts.append(f"Õpiväljundid: {row['oppivaaljundid_et']}")
    if pd.notna(row.get('oppivaaljundid_en')): parts.append(f"Learning outcomes: {row['oppivaaljundid_en']}")
    if pd.notna(row.get('hindamine_info')): parts.append(f"Hindamine: {row['hindamine_info']}")
    if pd.notna(row.get('eeldusained')): parts.append(f"Eeldusained: {row['eeldusained']}")
    return "\n".join(parts)

df_final['kirjeldus'] = df_final.apply(build_kirjeldus, axis=1)
df_final['kirjeldus_pikkus'] = df_final['kirjeldus'].str.len()

print("\nTunnuse 'kirjeldus' tähemärkide arvu statistika:")
print(df_final['kirjeldus_pikkus'].describe())

print(f"\nNäide (esimene aine):")
print(df_final['kirjeldus'].iloc[0][:500])

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(10, 5))
df_final['kirjeldus_pikkus'].hist(bins=50, ax=ax, edgecolor='black')
ax.set_xlabel('Tähemärkide arv')
ax.set_ylabel('Ainete arv')
ax.set_title('Tunnuse "kirjeldus" pikkuse jaotus')
ax.axvline(df_final['kirjeldus_pikkus'].median(), color='red', linestyle='--', label=f"Mediaan: {df_final['kirjeldus_pikkus'].median():.0f}")
ax.axvline(df_final['kirjeldus_pikkus'].mean(), color='orange', linestyle='--', label=f"Keskmine: {df_final['kirjeldus_pikkus'].mean():.0f}")
ax.legend()
plt.tight_layout()
plt.show()

df_final.drop(columns=['kirjeldus_pikkus']).to_csv(OUTPUT_FILE, index=False)
print(f"Lõplik andmestik salvestatud faili '{OUTPUT_FILE}' ({len(df_final)} rida, {len(df_final.columns) - 1} veergu).")